# 🎯 Netflix Hybrid Recommender with Feature Explanation


This notebook implements a hybrid recommender system that prioritizes:
- Genre matching from user input (mapped to real genres)
- Description similarity using TF-IDF
- Recency (release year)
- Content type (Movie or TV Show)


## 📥 Load and Preprocess Data

All data from the uploaded dataset is loaded and preprocessed.



We clean and prepare the Netflix dataset, converting types and filling missing values.


In [ ]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

df = pd.read_csv("netflix_titles.csv")
df['description'] = df['description'].fillna('')
df['listed_in'] = df['listed_in'].fillna('')
df['rating'] = df['rating'].fillna('Unknown')
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')
df = df.dropna(subset=['release_year'])
df['type_encoded'] = df['type'].map({'Movie': 1, 'TV Show': 0})


## 🔑 Genre Keyword Mapping

In [ ]:

keyword_expansion = {
    "romcom": ["Romantic Movies", "Comedies", "International Movies"],
    "action": ["Action & Adventure", "TV Action & Adventure"],
    "drama": ["Dramas", "TV Dramas"],
    "sci-fi": ["Sci-Fi & Fantasy", "TV Sci-Fi & Fantasy"],
    "crime": ["Crime TV Shows", "Crime Movies"],
    "thriller": ["Thrillers", "TV Thrillers"],
    "fantasy": ["Fantasy Movies", "TV Sci-Fi & Fantasy"],
    "romance": ["Romantic Movies", "Romantic TV Shows"],
    "comedy": ["Comedies", "TV Comedies"],
    "documentary": ["Documentaries", "Docuseries"]
}


this cell of code helps in relevance of the keyword used to search and reccommend on basis of genre searched

## 🧠 Define Hybrid Recommender

the hybrid recommender is being define in the code cell below.

In [ ]:

def recommend_advanced(user_keyword, preferred_type='Any', min_year=2000, max_year=2025, top_n=10):
    genre_keywords = keyword_expansion.get(user_keyword.lower(), [])
    
    filtered_df = df[df['listed_in'].apply(lambda g: any(k in g for k in genre_keywords))]
    if filtered_df.empty:
        return pd.DataFrame(columns=['title', 'listed_in', 'release_year', 'rating', 'type', 'description'])

    filtered_df = filtered_df[(filtered_df['release_year'] >= min_year) & (filtered_df['release_year'] <= max_year)]

    if preferred_type in ['Movie', 'TV Show']:
        filtered_df = filtered_df[filtered_df['type'] == preferred_type]

    if filtered_df.empty:
        return pd.DataFrame(columns=['title', 'listed_in', 'release_year', 'rating', 'type', 'description'])

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(filtered_df['description'])
    user_vector = tfidf.transform([" ".join(genre_keywords) + " " + user_keyword])
    desc_scores = linear_kernel(user_vector, tfidf_matrix).flatten()

    input_genres = set(genre_keywords)
    genre_scores = filtered_df['listed_in'].apply(lambda g: len(input_genres & set(g.split(", "))) / len(input_genres))

    max_year_data = df['release_year'].max()
    recency_scores = (filtered_df['release_year'] - min_year) / (max_year_data - min_year)

    if preferred_type in ['Movie', 'TV Show']:
        type_scores = (filtered_df['type'] == preferred_type).astype(int)
    else:
        type_scores = pd.Series([1] * len(filtered_df), index=filtered_df.index)

    final_scores = (
        0.4 * genre_scores +
        0.3 * desc_scores +
        0.2 * recency_scores +
        0.1 * type_scores
    )

    top_indices = final_scores.argsort()[-top_n:][::-1]
    return filtered_df.iloc[top_indices][['title', 'listed_in', 'release_year', 'rating', 'type', 'description']]


## ▶️ Try an Example

a test to check if the recommender is working


In [ ]:

recommend_advanced("romcom", preferred_type="Movie", min_year=2010, max_year=2025)



This output returns top genre-relevant Netflix titles that align with the user's keyword, year, and content preferences.
